In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline
import csv as csv
from scipy import stats

In [3]:
### Importing the data
data = pd.read_csv("/Users/saisravith/Downloads/train.csv")
data = data.set_index('ID')

In [358]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 76020 entries, 1 to 151838
Columns: 370 entries, var3 to TARGET
dtypes: float64(111), int64(259)
memory usage: 215.2 MB


In [359]:
data.describe()

,var3,var15,imp_ent_var16_ult1,imp_op_var39_comer_ult1,imp_op_var39_comer_ult3,imp_op_var40_comer_ult1,imp_op_var40_comer_ult3,imp_op_var40_efect_ult1,imp_op_var40_efect_ult3,imp_op_var40_ult1,...,saldo_medio_var33_hace2,saldo_medio_var33_hace3,saldo_medio_var33_ult1,saldo_medio_var33_ult3,saldo_medio_var44_hace2,saldo_medio_var44_hace3,saldo_medio_var44_ult1,saldo_medio_var44_ult3,var38,TARGET
count,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,...,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,7.602000e+04,76020.000000
mean,-1523.199277,33.212865,86.208265,72.363067,119.529632,3.559130,6.472698,0.412946,0.567352,3.160715,...,7.935824,1.365146,12.215580,8.784074,31.505324,1.858575,76.026165,56.614351,1.172358e+05,0.039569
std,39033.462364,12.956486,1614.757313,339.315831,546.266294,93.155749,153.737066,30.604864,36.513513,95.268204,...,455.887218,113.959637,783.207399,538.439211,2013.125393,147.786584,4040.337842,2852.579397,1.826646e+05,0.194945
min,-999999.000000,5.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.163750e+03,0.000000
25%,2.000000,23.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.787061e+04,0.000000
50%,2.000000,28.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.064092e+05,0.000000
75%,2.000000,40.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.187563e+05,0.000000
max,238.000000,105.000000,210000.000000,12888.030000,21024.810000,8237.820000,11073.570000,6600.000000,6600.000000,8237.820000,...,50003.880000,20385.720000,138831.630000,91778.730000,438329.220000,24650.010000,681462.900000,397884.300000,2.203474e+07,1.000000


Removing Columns based on Correlation

In [4]:
### Correlation matrix
correlation_input = data[data.columns.difference(["TARGET"])].corr(method = "pearson", min_periods = 1)

In [5]:
### Retrieving highly correlated input pairs (>0.8)
temp = correlation_input[(correlation_input[:]>0.8) & (correlation_input[:]!=1)]
temp.dropna(axis = 1,how = "all",inplace = True)

In [40]:
### Get column names
column_names = list(temp.columns.values)

In [41]:
len(column_names)

270

In [42]:
### Create dictionary of column names with correlated columns
ls = {}
for i in column_names:
    ls.update({i :temp[temp[i]>0].index.tolist()})

In [43]:
### Get column names that are important
column_names2 = column_names
for k,v in ls.items():
    if k in column_names2:
        for val in v:
            if val in column_names2 :
                column_names2.remove(val)

In [44]:
len(column_names2)

88

In [45]:
### Get column names that are to be removed
actual_column_names = list(data.columns.values)
columns_tobermoved = [i for i in actual_column_names if i not in column_names2]


In [46]:
### Drop the columns
for i in columns_tobermoved:
    if i != "TARGET":
        data.drop(i,axis = 1,inplace = True)

In [47]:
### Get low correlated input output pairs
correlation_input = data.corr(method = "pearson", min_periods = 1)
vals = correlation_input[abs(correlation_input["TARGET"])<0.0005]["TARGET"].index.tolist()

In [48]:
### Remove the columns
columns_tobermoved = [i for i in vals if i in list(data.columns.values)]
for i in columns_tobermoved:
    data.drop(i,axis = 1,inplace = True)

 Removing Outliers

In [424]:
### Outlier removal
## No outliers ith 3 standard deviation 0.1%
#data = data[(np.abs(stats.zscore(data)) < 3).all(axis = 1)]

## Outliers removal with 2.3 sd 1%
#data = data[(np.abs(stats.zscore(data)) < 2.3).all(axis = 1)]

Machine Learning

In [50]:
### 
nonremoved_list = list(data.columns.values)
vals = data.values

In [53]:
### Input and Target Separation 
input_series = data[data.columns.difference(["TARGET"])]
target_series = data["TARGET"]
input_data = input_series.values
target_data = target_series.values

In [79]:
type(target_data)

numpy.ndarray

In [81]:
len(input_data[0])

84

In [54]:
### Reading Test
temp = pd.read_csv("/Users/saisravith/Downloads/test.csv")
temp = temp.set_index("ID")

actual_column_names = list(temp.columns.values)
columns_tobermoved = [i for i in actual_column_names if i not in nonremoved_list]

for i in columns_tobermoved:
    temp.drop(i,axis = 1,inplace = True)
test_data = temp.values

In [373]:
#[i for i in list(temp.columns.values) if i not in nonremoved_list ]
#len(nonremoved_list)
#len(actual_column_names) - len(columns_tobermoved)
#len([ i for i in list(actual_column_names) if i not in columns_tobermoved])

Random Forest

In [56]:
### Importing Random Forest Libraries
from sklearn.cross_validation import cross_val_score
from sklearn.datasets import make_blobs
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier

In [62]:
### Fitting and Predicting
clf = RandomForestClassifier(n_estimators=10)
clf = clf.fit(input_data, target_data)
op = clf.predict(test_data)

In [63]:
stats.describe(op)

DescribeResult(nobs=75818, minmax=(0, 1), mean=1.3189480070695613e-05, variance=1.3189480070695616e-05, skewness=275.34523786183337, kurtosis=75813.00001318962)

In [64]:
out = pd.DataFrame(op,columns = ["Target"])

In [65]:
len(out[out["Target"]==1])

1

In [66]:
len(out[out["Target"]==0])

75817

In [55]:
type(out)

NameError: name 'out' is not defined

Support Vector Machine

In [75]:
len(data.columns)

85

In [ ]:
from sklearn import cross_validation
from sklearn.svm import SVC
clf = SVC()
k_fold = cross_validation.KFold(n=len(input_data), n_folds=10)
#for train_indices, test_indices in k_fold:
#    print('Train: %s | test: %s' % (train_indices, test_indices))
[svc.fit(input_data[train], target_data[train]).score(input_data[test], target_data[test])
    for train, test in k_fold]

In [87]:
from sklearn import cross_validation
k_fold = cross_validation.KFold(n=len(input_data), n_folds=10)
for train_indices, test_indices in k_fold:
    print('Train: %s | test: %s' % (train_indices, test_indices))

Train: [ 7602  7603  7604 ..., 76017 76018 76019] | test: [   0    1    2 ..., 7599 7600 7601]
Train: [    0     1     2 ..., 76017 76018 76019] | test: [ 7602  7603  7604 ..., 15201 15202 15203]
Train: [    0     1     2 ..., 76017 76018 76019] | test: [15204 15205 15206 ..., 22803 22804 22805]
Train: [    0     1     2 ..., 76017 76018 76019] | test: [22806 22807 22808 ..., 30405 30406 30407]
Train: [    0     1     2 ..., 76017 76018 76019] | test: [30408 30409 30410 ..., 38007 38008 38009]
Train: [    0     1     2 ..., 76017 76018 76019] | test: [38010 38011 38012 ..., 45609 45610 45611]
Train: [    0     1     2 ..., 76017 76018 76019] | test: [45612 45613 45614 ..., 53211 53212 53213]
Train: [    0     1     2 ..., 76017 76018 76019] | test: [53214 53215 53216 ..., 60813 60814 60815]
Train: [    0     1     2 ..., 76017 76018 76019] | test: [60816 60817 60818 ..., 68415 68416 68417]
Train: [    0     1     2 ..., 68415 68416 68417] | test: [68418 68419 68420 ..., 76017 76018 760

In [ ]:
clf.predict(test_data)

In [ ]:
from sklearn.svm import SVC
clf = SVC()
for i in range(10):
    X = train
    clf = clf.fit(X, y)

print(clf.predict([[-0.8, -1]]))